<a href="https://www.kaggle.com/code/amudheswaran/amx-vis-ablas-dick?scriptVersionId=134461170" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Amazon review spam detection

# Load data utilizing Apache SPARK 

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
            .appName('Amazon Review Spam Detection') \
            .getOrCreate()

# Load the JSON dataset
# df = spark.read.json("/kaggle/input/amazon-product-review-spam-and-non-spam/Electronics/Electronics.json")
df = spark.read.json("/kaggle/input/amazon-product-review-spam-and-non-spam/part.json/part.json")
# Show the first few rows of the DataFrame
df.show()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 14.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=106367ecf2821ab37bf1fadb81dde7cef1b8034088e8997e59d898d15c3982a1
  Stored in directory: /root/.cache/pip/wheels/06/51/98/f7a41aad64c08302d6c26c90650e713c3dfeb5cdec4946db00
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/22 08:14:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|BehaviouralFeatureResult|                 _id|      asin|            category|     helpful|label|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|    {0.0, 0.525950681...|{5a421d732bfbb88f...|B000809PCG|      Toys_and_Games|  [1.0, 2.0]|  0.0|    3.0|it's ok, but not ...| 01 9, 2013|A3ITLRYWLKJP5B|               Barry|Not the first gam...|    1357689600|
|    {0.0, 0.0, 1.0, 0...|{5a421d732bfbb88f...|B004Y6F9KW| Sports_and_Outdoors|  [0.0, 0.0]|  0.0|    5.0|I love this Air-s...|08 25, 2011| AA65S7IF7V9G

# Data preprocessing

* ## Understanding the datatypes for each vector

In [2]:
df.printSchema()


root
 |-- BehaviouralFeatureResult: struct (nullable = true)
 |    |-- AW: double (nullable = true)
 |    |-- CS: double (nullable = true)
 |    |-- ER: double (nullable = true)
 |    |-- FR: double (nullable = true)
 |    |-- MNR: double (nullable = true)
 |    |-- NR: double (nullable = true)
 |    |-- PC: double (nullable = true)
 |    |-- PR: double (nullable = true)
 |    |-- RB: double (nullable = true)
 |    |-- RC: double (nullable = true)
 |    |-- RD: double (nullable = true)
 |    |-- RL: double (nullable = true)
 |    |-- RSP: double (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- label: double (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullabl

* ## Checking for Null Values: Count the number of null values in each column.

In [6]:
from pyspark.sql.functions import when, count, col

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()


+------------------------+---+----+--------+-------+-----+-------+----------+----------+----------+------------+-------+--------------+
|BehaviouralFeatureResult|_id|asin|category|helpful|label|overall|reviewText|reviewTime|reviewerID|reviewerName|summary|unixReviewTime|
+------------------------+---+----+--------+-------+-----+-------+----------+----------+----------+------------+-------+--------------+
|                       0|  0|   0|       0|      0|80190|      0|         0|         0|         0|         724|      0|             0|
+------------------------+---+----+--------+-------+-----+-------+----------+----------+----------+------------+-------+--------------+



In [7]:
df.show()

+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|BehaviouralFeatureResult|                 _id|      asin|            category|     helpful|label|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|    {0.0, 0.525950681...|{5a421d732bfbb88f...|B000809PCG|      Toys_and_Games|  [1.0, 2.0]|  0.0|    3.0|it's ok, but not ...| 01 9, 2013|A3ITLRYWLKJP5B|               Barry|Not the first gam...|    1357689600|
|    {0.0, 0.0, 1.0, 0...|{5a421d732bfbb88f...|B004Y6F9KW| Sports_and_Outdoors|  [0.0, 0.0]|  0.0|    5.0|I love this Air-s...|08 25, 2011| AA65S7IF7V9G

# Phase1 Data Cleaning

## I Removing special characters from id

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_replace


cleaned_df = df.withColumn("_id", col("_id").getField("$oid"))
cleaned_df = cleaned_df.withColumn("_id", regexp_replace(col("_id"), "[{}]", ""))

cleaned_df.show()


+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|BehaviouralFeatureResult|                 _id|      asin|            category|     helpful|label|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+------------------------+--------------------+----------+--------------------+------------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|    {0.0, 0.525950681...|5a421d732bfbb88f1...|B000809PCG|      Toys_and_Games|  [1.0, 2.0]|  0.0|    3.0|it's ok, but not ...| 01 9, 2013|A3ITLRYWLKJP5B|               Barry|Not the first gam...|    1357689600|
|    {0.0, 0.0, 1.0, 0...|5a421d732bfbb88f1...|B004Y6F9KW| Sports_and_Outdoors|  [0.0, 0.0]|  0.0|    5.0|I love this Air-s...|08 25, 2011| AA65S7IF7V9G

## II Dropping unnecessary columns

In [ ]:
cleaned_df = cleaned_df.drop( 'category') #,'asin' , 'reviewerID', 'reviewerName') 
cleaned_df.show()

## III Ranmaing few Columns for QoL

In [9]:
cleaned_df = cleaned_df.withColumnRenamed("overall", "productRating")\
            .withColumnRenamed("_id", "id")
             


In [10]:
cleaned_df.show()

+------------------------+--------------------+----------+--------------------+------------+-----+-------------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|BehaviouralFeatureResult|                  id|      asin|            category|     helpful|label|productRating|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+------------------------+--------------------+----------+--------------------+------------+-----+-------------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|    {0.0, 0.525950681...|5a421d732bfbb88f1...|B000809PCG|      Toys_and_Games|  [1.0, 2.0]|  0.0|          3.0|it's ok, but not ...| 01 9, 2013|A3ITLRYWLKJP5B|               Barry|Not the first gam...|    1357689600|
|    {0.0, 0.0, 1.0, 0...|5a421d732bfbb88f1...|B004Y6F9KW| Sports_and_Outdoors|  [0.0, 0.0]|  0.0|          5.0|I love this Air-

## IV Placing the number of votes received as a metric from Helpful

In [11]:
cleaned_df = cleaned_df.withColumn('reviewUpvotes', col('helpful')[0])
cleaned_df.show()

+------------------------+--------------------+----------+--------------------+------------+-----+-------------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------+
|BehaviouralFeatureResult|                  id|      asin|            category|     helpful|label|productRating|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|reviewUpvotes|
+------------------------+--------------------+----------+--------------------+------------+-----+-------------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------+
|    {0.0, 0.525950681...|5a421d732bfbb88f1...|B000809PCG|      Toys_and_Games|  [1.0, 2.0]|  0.0|          3.0|it's ok, but not ...| 01 9, 2013|A3ITLRYWLKJP5B|               Barry|Not the first gam...|    1357689600|          1.0|
|    {0.0, 0.0, 1.0, 0...|5a421d732bfbb88f1...|B004Y6F9KW| Sports_and_Ou

## V Modifying the originally 'helpful' column to a float value showcasing No. of people who found it helpful among the total people that viewed it

In [12]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

def helpful_ratio(helpful):
    try:
        return helpful[0] / helpful[1]
    except ZeroDivisionError:
        return 0.0  # or any other value that makes sense in the context
    except:
        return None

# UDF registration
ratio_udf = udf(helpful_ratio, FloatType())

# Apply the UDF to the DataFrame
cleaned_df =  cleaned_df.withColumn('helpful', ratio_udf(col('helpful')))



In [13]:
cleaned_df =cleaned_df.withColumnRenamed("helpful", "helpfulTotalRatio")

In [14]:
cleaned_df.columns

['BehaviouralFeatureResult',
 'id',
 'asin',
 'category',
 'helpfulTotalRatio',
 'label',
 'productRating',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'summary',
 'unixReviewTime',
 'reviewUpvotes']

## VI Removing special characters from reviewText and convert to lowercase

In [15]:
from pyspark.sql.functions import col, lower, regexp_replace

cleaned_df = cleaned_df.withColumn("reviewText", lower(col("reviewText")))\
                 .withColumn("reviewText", regexp_replace(col("reviewText"), "[^\w\s]", ""))


In [ ]:
cleaned_df.show()

## VII Removing special characters from reviewText and convert to lowercase

In [ ]:

cleaned_df = cleaned_df.withColumn("summary", lower(col("summary")))\
                 .withColumn("summary", regexp_replace(col("summary"), "[^\w\s]", ""))


In [ ]:
cleaned_df.show()

## VIII Adding Additional Features

* ## Visualiztions for initial analysis of data

1. Distribution of Product Ratings

In [ ]:
import matplotlib.pyplot as plt

# Group by productRating and count the occurrences
rating_counts = cleaned_df.groupBy("productRating").count().orderBy("productRating")

# Convert to Pandas DataFrame for visualization
rating_counts_pd = rating_counts.toPandas()

# Create bar chart
plt.bar(rating_counts_pd["productRating"], rating_counts_pd["count"])
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Distribution of Ratings")
plt.show()


2. Finding the average rating of a product using asin for identifying distinct products. 

In [ ]:
from pyspark.sql.functions import avg

# Group the DataFrame by 'asin' and calculate the average rating
average_ratings = df.groupBy('asin').agg(avg('overall').alias('average_rating'))

# Show the average ratings of each product
average_ratings.show()


Lets Visualize the spam reviews and non spam reviews and see how many of them had helpfulTotalRatio column >0.5

In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame to include only reviews with helpfulTotalRatio > 0.5
helpful_reviews = cleaned_df.filter(col("helpfulTotalRatio") > 0.5)

# Group the DataFrame by the class column and count the number of reviews
spam_counts = cleaned_df.filter(col("class") == 1).count()
non_spam_counts = cleaned_df.filter(col("class") == 0).count()
helpful_counts = helpful_reviews.count()

# Create a bar plot to visualize the counts
x_values = ['Spam Reviews', 'Non-Spam Reviews', 'Helpful Reviews']
y_values = [spam_counts, non_spam_counts, helpful_counts]

plt.bar(x_values, y_values)
plt.xlabel('Review Type')
plt.ylabel('Count')
plt.title('Distribution of Spam and Non-Spam Reviews')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for spam reviews with helpfulTotalRatio > 0.5
spam_reviews = cleaned_df.filter((col("class") == 1) & (col("helpfulTotalRatio") > 0.5))

# Filter the DataFrame for non-spam reviews with helpfulTotalRatio > 0.5
non_spam_reviews = cleaned_df.filter((col("class") == 0) & (col("helpfulTotalRatio") > 0.5))

# Count the number of spam and non-spam reviews with helpfulTotalRatio > 0.5
spam_count = spam_reviews.count()
non_spam_count = non_spam_reviews.count()

# Create a bar plot
labels = ['Spam Reviews', 'Non-Spam Reviews']
counts = [spam_count, non_spam_count]

plt.bar(labels, counts)
plt.xlabel('Review Type')
plt.ylabel('Number of Reviews')
plt.title('Number of Reviews with helpfulTotalRatio > 0.5')

# Display the plot
plt.show()


* ## Some analysis for Ablation studies in the future

In [ ]:
from pyspark.sql.functions import col

duplicate_asin = df.groupBy("asin").count().filter(col("count") > 1)

# Show the duplicates
duplicate_asin.show()

In [ ]:
# Assuming your DataFrame is named "df"
duplicate_records = df.groupBy("asin", "reviewerID").count().filter(col("count") > 1)

# Show the duplicates
duplicate_records.show()

# EDA

In [ ]:
df["reviewerID"].value_counts()

# A.I violation if u use MFS

In [ ]:
# remove redundant columns
toys_df = toys.drop(['unixReviewTime', 'reviewerID', 'asin'], axis=1)
toys = []
toys_df = toys_df.dropna(subset=['class'])
# toys_df

# normalize and scale the overall column (product rating)
# x = toys_df['overall']
# x = x.values.reshape(-1,1) 
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# toys_df['overall'] = pd.DataFrame(x_scaled)
toys_df = toys_df.rename(columns={"overall": "rating", "helpful" : "votes-down/up"})
# toys_df['reviewText'] = toys_df['reviewText'].astype('str')

# id column cleanup
toys_df = toys_df.rename(columns={"_id": "id"})
toys_df['id'] = toys_df['id'].dropna()
toys_df['id'] = toys_df['id'].astype('str')
toys_df['id'] = toys_df['id'].str.split(' ').str[1].str.replace('}', '').str[1:-1]

# review time column cleanup
toys_df['reviewTime'] = toys_df['reviewTime'].str.replace(',', '').str.replace(' ', '.')

# text columns cleanup
toys_df['reviewText'] = toys_df['reviewText'].str.lower().str.replace('[^\w\s]', '')
toys_df = toys_df.loc[toys_df['reviewText'].str.len() > 90] # average length of the sentace is 20-25 words at 4.7 chars per word hence 90 characters or below is filtered

toys_df

## Feature engineering

In [ ]:
vectorization = TfidfVectorizer(analyzer = 'word', min_df=0.0, max_features=2000)
reviews = toys_df['reviewText'].to_list()
X = vectorization.fit_transform(reviews)

X_train, X_test, y_train, y_test = train_test_split(X, toys_df['class'], test_size = 0.2, random_state = 0)

In [ ]:
!apt-get install openjdk- 

## Model building

In [ ]:
def get_stats(y_pred, model_name):
  jacard = round(jaccard_score(y_test, y_pred, average='samples') * 100, 4)
  hamming = round(hamming_loss(y_pred, y_test) * 100, 4)
  print("Results for " + model_name)
  print("Jacard score: " + str(jacard))
  print("Hamming loss: " + str(hamming))
  print()

In [ ]:
lr = LogisticRegression(max_iter = 400)
mn = MultinomialNB()
svc = LinearSVC()
perc = Perceptron()

model_list = [lr, mn, svc, perc]

for model in model_list:
  model.fit(X_train, y_train)
  predictions = model.predict(X_test)
  cm = confusion_matrix(y_test, predictions)
  TN, FP, FN, TP = confusion_matrix(y_test, predictions).ravel()
  print(model.__class__.__name__)
  print('True Positive(TP)  = ', TP, np.round(TP / (TP+FP+TN+FN) * 100, 2))
  print('False Positive(FP) = ', FP, np.round(FP / (TP+FP+TN+FN) * 100, 2))
  print('True Negative(TN)  = ', TN, np.round(TN / (TP+FP+TN+FN) * 100, 2))
  print('False Negative(FN) = ', FN, np.round(FN / (TP+FP+TN+FN) * 100, 2))
  accuracy =  (TP+TN) /(TP+FP+TN+FN)
  print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm)  
  disp.plot(cmap=plt.cm.Blues)
  plt.show()
  print()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
svc = LinearSVC()
svc.fit(X_train, y_train)
predict = svc.predict(X_test)
figure(figsize=(15, 10))
matrix = confusion_matrix(y_test, predict)
sns.heatmap(matrix)
plt.show()
print(classification_report(y_test, predict))

# Unsupervised approach

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import silhouette_score
from tqdm import tqdm

scores = []
for i in tqdm(range(2, 4)):
  kmeans = KMeans(n_clusters=i, random_state=0)
  data = kmeans.fit_transform(X_train)
  label = kmeans.fit_predict(X_train)
  u_labels = np.unique(label)
  score = silhouette_score(X_train, kmeans.labels_, metric='euclidean')
  scores.append(score)

plt.plot(range(2,4), scores)
plt.title("KMeans best silhouette score")
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.show()

# Deep Learning

## Load and Preprocess Dataset

In [ ]:
import pandas as pd
import os
import torch
import transformers
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding, DistilBertTokenizerFast
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from tqdm.notebook import tqdm, trange
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
toys = pd.read_json('/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json', lines=True)

toys_df = toys.drop(['unixReviewTime', 'reviewerID', 'asin'], axis=1)
toys_df = toys_df.dropna(subset=['class'])

toys_df['reviewText'] = toys_df['reviewText'].str.lower().str.replace('[^\w\s]', '')
toys_df = toys_df.loc[toys_df['reviewText'].str.len() > 90]

In [ ]:
# Create Split
train_df, test_df = train_test_split(toys_df, test_size = 0.2, random_state = 0)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def convert_into_hugging_dataset(dataframe):
    new_df = dataframe[["class", "reviewText"]]
    new_df = new_df.rename(columns={"class": "label", "reviewText": "text"}) # convert into standard huggingface key names
    dataset = Dataset.from_pandas(new_df)
    return dataset

def preprocess_function(elements):
    return tokenizer(elements["text"], truncation=True)

dataset_train = convert_into_hugging_dataset(train_df)
dataset_test = convert_into_hugging_dataset(test_df)

tokenized_train = dataset_train.map(preprocess_function, batched=True)
tokenized_test = dataset_test.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Enables dynamic padding for batch

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

## Distilbert Model - BASIC

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=20000,
    save_steps=20000,
    weight_decay=0.01,
    load_best_model_at_end=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
os.makedirs("best_model", exist_ok=True)
trainer.save_model("best_model")
trainer.save_state()

## Evaluation

In [ ]:
trainer.evaluate()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("best_model_hyperopt", num_labels=2)

In [ ]:
pred_np = np.array([], dtype=int)
gt_np = np.array(test_df["class"].tolist(), dtype=int)
device = "cuda"

model = model.to(device)

batch_size = 16 # Adjust if you run into CUDA Out of Memory Exception or move to CPU
with torch.no_grad():
    for i in trange(0, len(test_df), batch_size):
        inputs = tokenizer(test_df["reviewText"][i:i + batch_size].tolist(), return_tensors="pt", padding=True, truncation=True)            
        labels = torch.tensor(test_df["class"][i:i + batch_size].tolist()).unsqueeze(1).to(device)
        inputs["input_ids"] = inputs["input_ids"].to(device)
        inputs["attention_mask"] = inputs["attention_mask"].to(device)

        outputs = model(**inputs, labels=labels)
        softmax = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(softmax, 1)
        pred_np = np.append(pred_np, pred_class.cpu().numpy())
        del inputs, labels, outputs, softmax, pred_class # -> ensure CUDA Memory is freed

In [ ]:
cm = confusion_matrix(gt_np, pred_np)
TN, FP, FN, TP = cm.ravel()
print("Deep Learning DistilBert Basic Model")
print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
accuracy =  (TP+TN) /(TP+FP+TN+FN)
print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))
print()

plt.figure()
disp = ConfusionMatrixDisplay(confusion_matrix=cm)  
disp.plot(cmap=plt.cm.Blues)
plt.savefig("evaluation_cm.png")
plt.show()
print(classification_report(gt_np, pred_np))

## Hyperparameter optimization

In [ ]:
!pip install optuna
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

def get_base_model():
    return AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# use only 10% of the training & evaluation data 
hyper_split = tokenized_train.shard(index=1, num_shards=10)
hyper_eval = tokenized_test.shard(index=1, num_shards=10)

hyper_train_args = TrainingArguments(
    output_dir="./hyperopt_bezospammer",
    learning_rate=5e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=20000,
    save_steps=20000,
    weight_decay=0.01)

hyper_trainer = Trainer(
    model_init=get_base_model,
    args=hyper_train_args,
    train_dataset=hyper_split,
    eval_dataset=hyper_eval,
    tokenizer=tokenizer)

In [ ]:
def get_hyperparamspace(trial):
    from ray import tune
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "num_train_epochs": tune.choice(list(range(1, 6))),
        "seed": tune.uniform(1, 40),
        "per_device_train_batch_size": tune.choice([4, 8, 12, 16]),
    }

best_run = hyper_trainer.hyperparameter_search(get_hyperparamspace, n_trials=15)

In [ ]:
best_run

In [ ]:
import json
with open("parameters.json", "w") as f:
    json.dump(dict(best_run.hyperparameters), f)

In [ ]:
training_args = TrainingArguments(
    output_dir="./hyper_opt_models",
    learning_rate= 1.717762111233842e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    evaluation_strategy="steps",
    eval_steps=20000,
    save_steps=20000,
    weight_decay=0.01,
    load_best_model_at_end=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# set seed since over trainer appy sets numpy seed as well bricks
setattr(trainer.args, "seed", 21.465500833657277)
trainer.args

In [ ]:
trainer.train()

In [ ]:
os.makedirs("best_model_hyperopt", exist_ok=True)
trainer.save_model("best_model_hyperopt")
trainer.save_state()

## Evaluation Different Category

In [ ]:
cell_phones = pd.read_json('/kaggle/input/amazon-product-review-spam-and-non-spam/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json', lines=True)
cell_phones = cell_phones.drop(['unixReviewTime', 'reviewerID', 'asin'], axis=1)
cell_phones = cell_phones.dropna(subset=['class'])

cell_phones['reviewText'] = cell_phones['reviewText'].str.lower().str.replace('[^\w\s]', '')
cell_phones = cell_phones.loc[cell_phones['reviewText'].str.len() > 90]

In [ ]:
pred_np = np.array([], dtype=int)
gt_np = np.array(cell_phones["class"].tolist(), dtype=int)
device = "cuda"

model = model.to(device)

batch_size = 16 # Adjust if you run into CUDA Out of Memory Exception or move to CPU
with torch.no_grad():
    for i in trange(0, len(cell_phones), batch_size):
        inputs = tokenizer(cell_phones["reviewText"][i:i + batch_size].tolist(), return_tensors="pt", padding=True, truncation=True)            
        labels = torch.tensor(cell_phones["class"][i:i + batch_size].tolist()).unsqueeze(1).to(device)
        inputs["input_ids"] = inputs["input_ids"].to(device)
        inputs["attention_mask"] = inputs["attention_mask"].to(device)

        outputs = model(**inputs, labels=labels)
        softmax = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(softmax, 1)
        pred_np = np.append(pred_np, pred_class.cpu().numpy())
        del inputs, labels, outputs, softmax, pred_class # -> ensure CUDA Memory is freed

In [ ]:
cm = confusion_matrix(gt_np, pred_np)
TN, FP, FN, TP = cm.ravel()
print("Deep Learning DistilBert Basic Model")
print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
accuracy =  (TP+TN) /(TP+FP+TN+FN)
print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))
print()

plt.figure()
disp = ConfusionMatrixDisplay(confusion_matrix=cm)  
disp.plot(cmap=plt.cm.Blues, values_format="d")
plt.show()
print(classification_report(gt_np, pred_np))